In [2]:
# Importing modules
import pandas as pd
import wget
import csv
import random
import os

In [3]:
# Creating tables

woman_names = []
man_names = []

woman_surnames = []
man_surnames = []


try:

# Downloading woman names from  https://dane.gov.pl/pl/dataset/1501,lista-imion-wystepujacych-w-rejestrze-pesel/resource/28099/table?page=1&per_page=20&q=&sort=    
    
    woman_names_url = r'https://bit.ly/3iXOZzs'
    woman_names_file = wget.download(woman_names_url)


    with open(woman_names_file, 'rt',encoding='utf-8' ,errors="ignore") as woman_names_csv:
        
        csv_read_woman = csv.DictReader(woman_names_csv)
        
        for row in csv_read_woman:
                woman_names.append(row['IMIĘ_PIERWSZE'])
                
    os.remove('3iXOZzs')

            
# Downloading man names from https://dane.gov.pl/pl/dataset/1501,lista-imion-wystepujacych-w-rejestrze-pesel/resource/28100/table?page=1&per_page=20&q=&sort=


    man_names_url = r'https://bit.ly/3qpdUgX'
    man_names_file = wget.download(man_names_url)


    with open(man_names_file, 'rt',encoding='utf-8' ,errors="ignore") as man_names_csv:
        
        csv_read_man = csv.DictReader(man_names_csv)
        
        for row in csv_read_man:
                man_names.append(row['IMIĘ_PIERWSZE'])
                
    os.remove('3qpdUgX')
            
# Downloading woman surnames from https://dane.gov.pl/pl/dataset/1681,nazwiska-osob-zyjacych-wystepujace-w-rejestrze-pesel/resource/28080/table?page=1&per_page=20&q=&sort=

    woman_surnames_url = r'https://bit.ly/3zMpBCK'
    woman_surnames_file = wget.download(woman_surnames_url)


    with open(woman_surnames_file, 'rt',encoding='utf-8' ,errors="ignore") as woman_surnames_csv:
        
        csv_read_woman_surnames = csv.DictReader(woman_surnames_csv)
        
        for row in csv_read_woman_surnames:
                woman_surnames.append(row['Nazwisko aktualne'])
                
    os.remove('3zMpBCK')

# Downloading man surnames from https://dane.gov.pl/pl/dataset/1681,nazwiska-osob-zyjacych-wystepujace-w-rejestrze-pesel/resource/28052/table?page=1&per_page=20&q=&sort=             

    man_surnames_url = r'https://bit.ly/3japaMq'
    man_surnames_file = wget.download(man_surnames_url)

    with open(man_surnames_file, 'rt',encoding='utf-8' ,errors="ignore") as man_surnames_csv:
        
        csv_read_man_surnames = csv.DictReader(man_surnames_csv)
        
        for row in csv_read_man_surnames:
                man_surnames.append(row['Nazwisko aktualne'])
                
    os.remove('3japaMq')
                
except ValueError:
    print('Files already exist!')
    pass


except KeyError:
    print('Wrong dictionary key')

In [4]:
        
def rand_consumer_list(num=1, man_possibility=1.0, woman_possibility=1.0, min_age=18, max_age=70):
    
    consumer_list = []
    x = 0
    
    while x < num:
        possibility_roll = random.random()
        
        if possibility_roll <= man_possibility:
            
            consumer_list.append([random.choice(man_names),
                                  random.choice(man_surnames),
                                  random.randint(min_age,max_age),
                                  "MĘŻCZYZNA"
                                 ])
                
                
        if possibility_roll <= woman_possibility:
            
            consumer_list.append([random.choice(woman_names),
                                  random.choice(woman_surnames),
                                  random.randint(min_age,max_age),
                                  "KOBIETA"
                                 ])
        x+=1
        
        
    print(consumer_list) 
    
rand_consumer_list(14)    

[['MAIKEL', 'MAKIŁŁO', 52, 'MĘŻCZYZNA'], ['LEANNA', 'DĘBAKOWSKA', 56, 'KOBIETA'], ['JUNNOSUKE', 'CATS', 65, 'MĘŻCZYZNA'], ['HUILI', 'GREIL', 22, 'KOBIETA'], ['QUOC NAM', 'KURJAŃSKI', 54, 'MĘŻCZYZNA'], ['BIRGITTA', 'LOS-PŁAWSZEWSKA', 48, 'KOBIETA'], ['MAKSYMIAN', 'KEREN ZVI', 46, 'MĘŻCZYZNA'], ['RADKHA', 'KABAKLI', 49, 'KOBIETA'], ['CHRISTIANN', 'FLATAU-KOWALSKI', 31, 'MĘŻCZYZNA'], ['ORYSTA', 'SYROWICZKA', 58, 'KOBIETA'], ['HONGXING', 'LIND', 47, 'MĘŻCZYZNA'], ['STAFANIA', 'STEFFKE', 64, 'KOBIETA'], ['PEDRO', 'HOLOVA', 56, 'MĘŻCZYZNA'], ['SULAMITA', 'SARRON', 42, 'KOBIETA'], ['KASIAN', 'GRACA', 59, 'MĘŻCZYZNA'], ['MITSUE', 'JÄCKEL', 21, 'KOBIETA'], ['WENANTY', 'ZAPISHNYI', 69, 'MĘŻCZYZNA'], ['AISTE', 'CHRISSIDOU', 22, 'KOBIETA'], ['KWANG YONG', 'TASIOR', 69, 'MĘŻCZYZNA'], ['MAKSI', 'ŞÜKÜR', 24, 'KOBIETA'], ['JERZY       TADEUSZ', 'SWIĘDZIOŁ', 38, 'MĘŻCZYZNA'], ['HUDA', 'MARTUŚ', 68, 'KOBIETA'], ['KEVIN', 'KHAMUTOU', 37, 'MĘŻCZYZNA'], ['ANGELLA', 'CHUPILKO', 18, 'KOBIETA'], ['ARNAS', 'CH